In [ ]:
import project.constants as cn
import project.utils.file_system as fs
import numpy as np
from scipy.interpolate import interp1d

In [1]:
movement_number = 1

n_subjects = 10  # num_of_exercises
n_episodes = 10  # numb_of_reputation_per_exersice

PREPARE_DATA_DIR = "../../../data_output"

UI_PRMD_DIR = '../datasets/ui-prmd/'
UI_PRMD_CORRECT_SEGMENTED_MOVEMENTS = '../dataset/Segmented\ Movements/'
UI_PRMD_INCORRECT_SEGMENTED_MOVEMENTS = '../dataset/SegmentedIncorrect\ Segmented\ Movements/'

UI_MOVEMENT_CORRECT_PATH = '../datasets/ui-prmd/Segmented Movements/'
UI_MOVEMENT_INCORRECT_PATH = '../datasets/ui-prmd/SegmentedIncorrect Segmented Movements/'


In [ ]:
def load_data(subject, episode, correct=True):
    prefix = f"{UI_MOVEMENT_CORRECT_PATH}/Vicon/Angles/" if correct else f"{UI_MOVEMENT_INCORRECT_PATH}/Vicon/Angles/"
    filename = f"m{movement_number:02d}_s{subject:02d}_e{episode:02d}_angles.txt"
    path = prefix + filename
    return np.loadtxt(path, delimiter=',')


def ensure_exercise_exists(ex_dir_name):
    fs.create_dir_if_not_exists(cn.UI_PRMD_CORRECT_SEGMENTED_MOVEMENTS)
    fs.ensure_dir_exists(ex_dir_name)


In [ ]:
# Load data for correct sequences

correct_data = [load_data(s, e) for s in range(1, n_subjects + 1)
                for e in range(1, n_episodes + 1)]

incorrect_data = [load_data(s, e, correct=False) for s in range(1, n_subjects + 1)
                  for e in range(1, n_episodes + 1)]

In [ ]:
# Perform linear alignment
length_mean = 240
correct_aligned = [interp1d(np.arange(len(seq)), seq, axis=0)(np.linspace(0, len(seq) - 1, length_mean)) for seq
                   in correct_data]
incorrect_aligned = [interp1d(np.arange(len(seq)), seq, axis=0)(np.linspace(0, len(seq) - 1, length_mean)) for
                     seq in incorrect_data]

In [ ]:
# Transform into matrices
correct_xm = np.concatenate(correct_aligned)
incorrect_xm = np.concatenate(incorrect_aligned)

In [ ]:
# Re-center the data to obtain zero mean
data_mean = np.mean(correct_xm, axis=1)
centered_correct_data = correct_xm - data_mean[:, np.newaxis]
data_mean_inc = np.mean(incorrect_xm, axis=1)
centered_incorrect_data = incorrect_xm - data_mean_inc[:, np.newaxis]

In [ ]:
# Scale the data between -1 and 1
scaling_value = np.max([np.max(centered_correct_data), np.max(np.abs(centered_correct_data))])
scaling_value = np.ceil(scaling_value)

data_correct = centered_correct_data / scaling_value
data_incorrect = centered_incorrect_data / scaling_value


In [ ]:
# Save the data
np.savetxt(f'{PREPARE_DATA_DIR}/Data_Correct.csv', data_correct, delimiter=',')
np.savetxt(f'{PREPARE_DATA_DIR}/Data_Incorrect.csv', data_incorrect, delimiter=',')
